# Polynomial Regression Models

## CONTENTS

__1.LIBRARIES AND DATA__
    
_Importing Necessary tools for data science and dataset_
        
__2.DEFINING FUNCTIONS__

_Writing functions that will be frequently used_

__3.ADDING POLYNOMIAL FEATURES__

_Expanding the number of predictors by adding Polynomial Features of_

__4.APPLYING FEATURE SELECTION METHODS TO POLYNOMIAL DATAFRAMES__

    4.1 Applying Features Selection Methods to X = X^1 + X^2 dataset
    4.2 Applying Features Selection Methods to X = X^1 + X^2 + X^3 dataset

__5.APPLYING THE SAME STRATEGY FROM PART 4 TO A REDUCED DATASET__

    5.1 Reduce the dataset via forward selection
    5.2 Add Polynomial features to the reduced dataset
    5.3 Run MLR models and store results
    5.4 Further reduce the dataset via all the feature selecting methods
    5.5 Run MLR models and store results
    
__6.CONCLUSION__
    

## 1. LIBRARIES AND DATA

__Libraries__

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', 100)

__Importing Data__

In [2]:
cancer_df = pd.read_csv(r'C:\Users\Constantine\OneDrive\Υπολογιστής\Regression Port\cancer_reg_refined.csv',
                 encoding='latin-1') 
cancer_df.head(2)

# Without the encoding paremeter, this error presents itself: UnicodeDecodeError: 'utf-8' codec can't decode 
# byte 0xf1 in position 41137: invalid continuation byte

,avgAnnCount,avgDeathsPerYear,TARGET_deathRate,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,binnedInc,MedianAge,MedianAgeMale,MedianAgeFemale,Geography,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate
0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,93564.75,39.3,36.9,41.7,WEST,2.54,52.5,11.5,39.5,6.9,23.2,19.6,51.9,8.0,75.1,41.6,32.9,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831
1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,49534.00,33.0,32.2,33.7,WEST,2.34,44.5,6.1,22.4,7.5,26.0,22.7,55.9,7.8,70.2,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096


In [3]:
# Copying and shuffling the original Dataframe
df1 = cancer_df.sample(frac = 1) 

In [4]:
# Getting Dummy variables
df1 = pd.get_dummies(data = df1, drop_first = True)
df1.head(2) 

,avgAnnCount,avgDeathsPerYear,TARGET_deathRate,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,binnedInc,MedianAge,MedianAgeMale,MedianAgeFemale,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,Geography_EAST,Geography_WEST
1942,895.0,395,182.7,431.1,42622,181877,17.7,87.971541,41543.55,37.8,36.5,39.3,2.35,46.6,13.2,29.9,8.8,34.5,15.8,59.2,6.2,68.4,50.7,31.7,17.6,86.153719,9.056562,1.071123,0.826232,43.728526,4.541089,0,0
2208,201.0,87,187.8,441.3,35180,37388,21.8,80.239649,35815.95,39.9,37.4,42.2,2.49,50.8,19.2,34.1,4.1,36.3,10.3,50.4,9.0,53.7,36.0,41.3,24.1,84.576221,11.471368,0.270640,1.248694,48.534378,4.523580,1,0


## 2. DEFINING FUNCTIONS 

In [5]:
# Defining forward selection function
def forward_selection(data, target, significance_level):
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<significance_level):
            best_features.append(new_pval.idxmin())
        else:
            break
    return best_features


# Defining backward elimination function
def backward_elimination(data, target,significance_level):
    features = data.columns.tolist()
    while(len(features)>0):
        features_with_constant = sm.add_constant(data[features])
        p_values = sm.OLS(target, features_with_constant).fit().pvalues[1:]
        max_p_value = p_values.max()
        if(max_p_value >= significance_level):
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break 
    return features


# Defining the stepwise selection function
def stepwise_selection(data, target,SL_in,SL_out):
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<SL_in):
            best_features.append(new_pval.idxmin())
            while(len(best_features)>0):
                best_features_with_constant = sm.add_constant(data[best_features])
                p_values = sm.OLS(target, best_features_with_constant).fit().pvalues[1:]
                max_p_value = p_values.max()
                if(max_p_value >= SL_out):
                    excluded_feature = p_values.idxmax()
                    best_features.remove(excluded_feature)
                else:
                    break 
        else:
            break
    return best_features


# Using the RepeatedKFold function for cross-validation
rkf = RepeatedKFold(n_splits= 5, n_repeats= 10, random_state= 126) 


# Creating function that calculates cross-validated scores
def cv_scores(X,Y):
        
    cv_MAE = round(np.mean(cross_val_score(LinearRegression(), X, Y, cv=rkf, 
                                       scoring='neg_mean_absolute_error', error_score='raise') * (-1)),3)

    cv_MSE = round(np.mean(cross_val_score(LinearRegression(), X, Y, cv=rkf, 
                                          scoring='neg_mean_squared_error', error_score='raise') * (-1)),1)
    
    cv_MAPE = round(np.mean(cross_val_score(LinearRegression(), X, Y, cv=rkf, 
                                          scoring='neg_mean_absolute_percentage_error', error_score='raise') * (-100)),3)
    
    cv_R2 = round(np.mean(cross_val_score(LinearRegression(), X, Y, cv=rkf, 
                                          scoring='r2', error_score='raise')),3)
    
    cv_F_stat = round(((cv_R2)/(1-cv_R2))*((len(X) - len(X.columns) - 1 )/len(X.columns)),2)
    
    score_list = [cv_MAE, cv_MSE,cv_MAPE,cv_R2,cv_F_stat,len(X.columns)]
    return score_list

## 3. ADDING POLYNOMIAL FEATURES 

In [6]:
# Defining X and Y

Y = df1['TARGET_deathRate']
X = df1.drop('TARGET_deathRate', axis = 1)
X.head(2)

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,binnedInc,MedianAge,MedianAgeMale,MedianAgeFemale,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,Geography_EAST,Geography_WEST
1942,895.0,395,431.1,42622,181877,17.7,87.971541,41543.55,37.8,36.5,39.3,2.35,46.6,13.2,29.9,8.8,34.5,15.8,59.2,6.2,68.4,50.7,31.7,17.6,86.153719,9.056562,1.071123,0.826232,43.728526,4.541089,0,0
2208,201.0,87,441.3,35180,37388,21.8,80.239649,35815.95,39.9,37.4,42.2,2.49,50.8,19.2,34.1,4.1,36.3,10.3,50.4,9.0,53.7,36.0,41.3,24.1,84.576221,11.471368,0.270640,1.248694,48.534378,4.523580,1,0


In [7]:
# Dividing X to numeric and categorical dataframes

X_num = X.drop(columns = ['Geography_EAST','Geography_WEST'])
X_cat = X[['Geography_EAST','Geography_WEST']]

# There is no point to addying categorical features to power of n to the dataset

__Creating the X^2 and X^3 dataframes__

In [8]:
# Copying the numeric dataframe and assigning it to a variable.
X2_num = X_num.copy()  

In [9]:
# Loop that adds polynomial feature of 2nd degree.
for i in X_num.columns:
    
    name2 = i + '^2'
    
    Z2 = X_num[i] * X_num[i]
    
    X2_num[name2] = Z2

In [10]:
# Copying the numeric dataframe and assigning it to a variable.
X3_num = X2_num.copy()

In [11]:
# Loop that adds polynomial feature of 3rd degree.
for i in X_num.columns:
    
    name3 = i + '^3'
    
    Z3 = X_num[i] * X_num[i] * X_num[i]
   
    X3_num[name3] = Z3

In [12]:
# Concatenating numeric and categorical dataframes
X2 = pd.concat([X2_num.round(3), X_cat], axis =  1)
X3 = pd.concat([X3_num.round(3), X_cat], axis =  1)

In [13]:
# Independent dataframe is X + X^2
X2.head(2)

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,binnedInc,MedianAge,MedianAgeMale,MedianAgeFemale,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,avgAnnCount^2,avgDeathsPerYear^2,incidenceRate^2,medIncome^2,popEst2015^2,povertyPercent^2,studyPerCap^2,binnedInc^2,MedianAge^2,MedianAgeMale^2,MedianAgeFemale^2,AvgHouseholdSize^2,PercentMarried^2,PctNoHS18_24^2,PctHS18_24^2,PctBachDeg18_24^2,PctHS25_Over^2,PctBachDeg25_Over^2,PctEmployed16_Over^2,PctUnemployed16_Over^2,PctPrivateCoverage^2,PctEmpPrivCoverage^2,PctPublicCoverage^2,PctPublicCoverageAlone^2,PctWhite^2,PctBlack^2,PctAsian^2,PctOtherRace^2,PctMarriedHouseholds^2,BirthRate^2,Geography_EAST,Geography_WEST
1942,895.0,395,431.1,42622,181877,17.7,87.972,41543.55,37.8,36.5,39.3,2.35,46.6,13.2,29.9,8.8,34.5,15.8,59.2,6.2,68.4,50.7,31.7,17.6,86.154,9.057,1.071,0.826,43.729,4.541,801025.0,156025,185847.21,1816634884,33079243129,313.29,7738.992,1.725867e+09,1428.84,1332.25,1544.49,5.523,2171.56,174.24,894.01,77.44,1190.25,249.64,3504.64,38.44,4678.56,2570.49,1004.89,309.76,7422.463,82.021,1.147,0.683,1912.184,20.621,0,0
2208,201.0,87,441.3,35180,37388,21.8,80.240,35815.95,39.9,37.4,42.2,2.49,50.8,19.2,34.1,4.1,36.3,10.3,50.4,9.0,53.7,36.0,41.3,24.1,84.576,11.471,0.271,1.249,48.534,4.524,40401.0,7569,194745.69,1237632400,1397862544,475.24,6438.401,1.282782e+09,1592.01,1398.76,1780.84,6.200,2580.64,368.64,1162.81,16.81,1317.69,106.09,2540.16,81.00,2883.69,1296.00,1705.69,580.81,7153.137,131.592,0.073,1.559,2355.586,20.463,1,0


In [14]:
# Independent dataframe is X + X^2 + X^3
X3.head(2)

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,binnedInc,MedianAge,MedianAgeMale,MedianAgeFemale,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,avgAnnCount^2,avgDeathsPerYear^2,incidenceRate^2,medIncome^2,popEst2015^2,povertyPercent^2,studyPerCap^2,binnedInc^2,MedianAge^2,MedianAgeMale^2,MedianAgeFemale^2,AvgHouseholdSize^2,PercentMarried^2,PctNoHS18_24^2,PctHS18_24^2,PctBachDeg18_24^2,PctHS25_Over^2,PctBachDeg25_Over^2,PctEmployed16_Over^2,PctUnemployed16_Over^2,PctPrivateCoverage^2,PctEmpPrivCoverage^2,PctPublicCoverage^2,PctPublicCoverageAlone^2,PctWhite^2,PctBlack^2,PctAsian^2,PctOtherRace^2,PctMarriedHouseholds^2,BirthRate^2,avgAnnCount^3,avgDeathsPerYear^3,incidenceRate^3,medIncome^3,popEst2015^3,povertyPercent^3,studyPerCap^3,binnedInc^3,MedianAge^3,MedianAgeMale^3,MedianAgeFemale^3,AvgHouseholdSize^3,PercentMarried^3,PctNoHS18_24^3,PctHS18_24^3,PctBachDeg18_24^3,PctHS25_Over^3,PctBachDeg25_Over^3,PctEmployed16_Over^3,PctUnemployed16_Over^3,PctPrivateCoverage^3,PctEmpPrivCoverage^3,PctPublicCoverage^3,PctPublicCoverageAlone^3,PctWhite^3,PctBlack^3,PctAsian^3,PctOtherRace^3,PctMarriedHouseholds^3,BirthRate^3,Geography_EAST,Geography_WEST
1942,895.0,395,431.1,42622,181877,17.7,87.972,41543.55,37.8,36.5,39.3,2.35,46.6,13.2,29.9,8.8,34.5,15.8,59.2,6.2,68.4,50.7,31.7,17.6,86.154,9.057,1.071,0.826,43.729,4.541,801025.0,156025,185847.21,1816634884,33079243129,313.29,7738.992,1.725867e+09,1428.84,1332.25,1544.49,5.523,2171.56,174.24,894.01,77.44,1190.25,249.64,3504.64,38.44,4678.56,2570.49,1004.89,309.76,7422.463,82.021,1.147,0.683,1912.184,20.621,716917375.0,61629875,8.011873e+07,77428612025848,6016353502573133,5545.233,680811.059,7.169862e+13,54010.152,48627.125,60698.457,12.978,101194.696,2299.968,26730.899,681.472,41063.625,3944.312,207474.688,238.328,320013.504,130323.843,31855.013,5451.776,639472.814,742.831,1.229,0.564,83616.986,93.644,0,0
2208,201.0,87,441.3,35180,37388,21.8,80.240,35815.95,39.9,37.4,42.2,2.49,50.8,19.2,34.1,4.1,36.3,10.3,50.4,9.0,53.7,36.0,41.3,24.1,84.576,11.471,0.271,1.249,48.534,4.524,40401.0,7569,194745.69,1237632400,1397862544,475.24,6438.401,1.282782e+09,1592.01,1398.76,1780.84,6.200,2580.64,368.64,1162.81,16.81,1317.69,106.09,2540.16,81.00,2883.69,1296.00,1705.69,580.81,7153.137,131.592,0.073,1.559,2355.586,20.463,8120601.0,658503,8.594127e+07,43539907832000,52263284795072,10360.232,516615.060,4.594407e+13,63521.199,52313.624,75151.448,15.438,131096.512,7077.888,39651.821,68.921,47832.147,1092.727,128024.064,729.000,154854.153,46656.000,70444.997,13997.521,604985.314,1509.544,0.020,1.947,114326.894,92.565,1,0


In [15]:
# Storing the cross-validated scores

CV_RESULTS = pd.DataFrame({'Pol.F_2deg':cv_scores(X2,Y)}, index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])


CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'Pol.F_3deg':cv_scores(X3,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS

,Pol.F_2deg,Pol.F_3deg
MAE,13.372,25.257
MSE,333.500,1720.900
MAPE,7.756,14.758
R^2,0.562,-1.257
F-stat,54.160,-15.660
N_Preds,62.000,92.000


## 4. APPLYING FEATURE SELECTION METHODS TO POLYNOMIAL DATAFRAMES

__4.1 Applying Features Selection Methods to X = X^1 + X^2 dataset__

In [16]:
# Reducing X with Forward Selection
X2_f = X2[forward_selection(X2,Y,0.01)]

In [17]:
X2_f.head(2)

,PctBachDeg25_Over,incidenceRate,povertyPercent,Geography_WEST,PctHS18_24,PctPrivateCoverage^2,PctOtherRace,PctEmpPrivCoverage^2,PctEmployed16_Over^2,MedianAgeFemale^2,PctMarriedHouseholds,PercentMarried,BirthRate,PctOtherRace^2,avgAnnCount,medIncome^2,PctPrivateCoverage,PctPublicCoverageAlone^2,PctPublicCoverageAlone,PctBlack^2,PctWhite,medIncome
1942,15.8,431.1,17.7,0,29.9,4678.56,0.826,2570.49,3504.64,1544.49,43.729,46.6,4.541,0.683,895.0,1816634884,68.4,309.76,17.6,82.021,86.154,42622
2208,10.3,441.3,21.8,0,34.1,2883.69,1.249,1296.00,2540.16,1780.84,48.534,50.8,4.524,1.559,201.0,1237632400,53.7,580.81,24.1,131.592,84.576,35180


In [18]:
# Reducing X with Backward Elimination
X2_b = X2[backward_elimination(X2,Y,0.01)]

In [19]:
X2_b.head(2)

,avgAnnCount,avgDeathsPerYear,incidenceRate,popEst2015,binnedInc,PercentMarried,PctHS18_24,PctBachDeg25_Over,PctEmployed16_Over,PctPrivateCoverage,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctOtherRace,PctMarriedHouseholds,BirthRate,avgAnnCount^2,avgDeathsPerYear^2,medIncome^2,popEst2015^2,binnedInc^2,MedianAge^2,PercentMarried^2,PctPrivateCoverage^2,PctEmpPrivCoverage^2,PctPublicCoverageAlone^2,PctWhite^2,PctBlack^2,PctOtherRace^2,PctMarriedHouseholds^2,Geography_WEST
1942,895.0,395,431.1,181877,41543.55,46.6,29.9,15.8,59.2,68.4,50.7,31.7,17.6,86.154,0.826,43.729,4.541,801025.0,156025,1816634884,33079243129,1.725867e+09,1428.84,2171.56,4678.56,2570.49,309.76,7422.463,82.021,0.683,1912.184,0
2208,201.0,87,441.3,37388,35815.95,50.8,34.1,10.3,50.4,53.7,36.0,41.3,24.1,84.576,1.249,48.534,4.524,40401.0,7569,1237632400,1397862544,1.282782e+09,1592.01,2580.64,2883.69,1296.00,580.81,7153.137,131.592,1.559,2355.586,0


In [20]:
# Reducing X with Stepwise Selection
X2_s = X2[stepwise_selection(X2,Y,0.01,0.01)]

In [21]:
X2_s.head(2)

,PctBachDeg25_Over,incidenceRate,Geography_WEST,PctHS18_24,PctPrivateCoverage^2,PctOtherRace,PctEmpPrivCoverage^2,PctEmployed16_Over^2,MedianAgeFemale^2,PctMarriedHouseholds,PercentMarried,BirthRate,PctOtherRace^2,avgAnnCount
1942,15.8,431.1,0,29.9,4678.56,0.826,2570.49,3504.64,1544.49,43.729,46.6,4.541,0.683,895.0
2208,10.3,441.3,0,34.1,2883.69,1.249,1296.00,2540.16,1780.84,48.534,50.8,4.524,1.559,201.0


In [22]:
# Storing the cross-validated scores

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'2deg_reduced_by_f':cv_scores(X2_f,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'2deg_reduced_by_b':cv_scores(X2_b,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'2deg_reduced_by_s':cv_scores(X2_s,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS

,Pol.F_2deg,Pol.F_3deg,2deg_reduced_by_f,2deg_reduced_by_b,2deg_reduced_by_s
MAE,13.372,25.257,14.036,13.227,14.163
MSE,333.500,1720.900,359.800,324.400,367.500
MAPE,7.756,14.758,8.112,7.662,8.181
R^2,0.562,-1.257,0.527,0.574,0.517
F-stat,54.160,-15.660,134.560,111.460,203.760
N_Preds,62.000,92.000,22.000,32.000,14.000


__4.2 Applying Features Selection Methods to X = X^1 + X^2 + X^3 dataset__

In [23]:
# Reducing X with Forward Selection
X3_f = X3[forward_selection(X3,Y, 0.005)]

In [24]:
# Reducing X with Backward Elimination
X3_b = X3[backward_elimination(X3,Y,0.005)]

We will not be using the stepwise function because the calculation time is quite large!

In [25]:
# Storing the cross-validated scores

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'3deg_reduced_by_f':cv_scores(X3_f,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'3deg_reduced_by_b':cv_scores(X3_b,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)


CV_RESULTS

KeyboardInterrupt: 

## 5. APPLYING THE SAME STRATEGY FROM PART 4 TO A REDUCED DATASET

We remember that if we applied any feature selection methods to the original dataset, we would end with the same predictors.
The strategy that we are going to apply is:

-  Step 1: Reduce the dataset via forward selection
-  Step 2: Add Polynomial features to the reduced dataset
-  Step 3: Run MLR models and store results
-  Step 4: Further reduce the dataset via all the feature selecting methods
-  Step 5: Run MLR models and store results

__5.1. Reduce the dataset via forward selection__

In [ ]:
# Reducing the dataset via forward selection
X_f = X[forward_selection(X,Y, 0.01)]

In [ ]:
X_f.head(2)

__5.2. Addying Polynomial Features to the reduced dataset__

In [ ]:
# Copying the dataframe
X_f_2 = X_f.copy()

In [ ]:
for i in X_f.columns:
    
    name2 = i + '^2'
    
    Z2 = X_f[i] * X_f[i]
    
    X_f_2[name2] = Z2

In [ ]:
# Independent dataframe is X + X^2
X_f_2.head(2)

In [ ]:
# Deleting the redundant variables
X_f_2 = X_f_2.drop('Geography_WEST^2', axis = 1)

In [ ]:
# Checking again the Independent dataframe
X_f_2.head(2)

In [ ]:
# Copying the dataframe
X_f_3 = X_f_2.copy()

In [ ]:
for i in X_f.columns:
    
    name3 = i + '^3'
    
    Z3 = X_f[i] * X_f[i] * X_f[i]
   
    X_f_3[name3] = Z3

In [ ]:
# Independent dataframe is X + X^2 + X^3
X_f_3.head(2)

In [ ]:
# Deleting the redundant variables
X_f_3 = X_f_3.drop('Geography_WEST^3', axis = 1)

In [ ]:
# Checking again the Independent dataframe
X_f_3.head(2)

__5.3. Running MLR models and storing results__

In [ ]:
# Storing the cross-validated scores

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_2deg':cv_scores(X_f_2,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_3deg':cv_scores(X_f_3,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)


CV_RESULTS

__5.4. Further reduce the dataset via all the feature selecting methods__

In [ ]:
# Reducing the Reduced Dataset with Polynomial Features of 2nd degree via forward selection
X_f_2_f = X_f_2[forward_selection(X_f_2,Y, 0.01)]

In [ ]:
# Reducing the Reduced Dataset with Polynomial Features of 2nd degree via backward elimination
X_f_2_b = X_f_2[backward_elimination(X_f_2,Y,0.01)]

In [ ]:
# Reducing the Reduced Dataset with Polynomial Features of 2nd degree via stepwise selection
X_f_2_s = X_f_2[stepwise_selection(X_f_2,Y,0.01, 0.01)]

In [ ]:
# Storing the cross-validated scores

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_2deg_reduced_f':cv_scores(X_f_2_f,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_2deg_reduced_b':cv_scores(X_f_2_b,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_2deg_reduced_s':cv_scores(X_f_2_s,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS

In [ ]:
# Reducing the Reduced Dataset with Polynomial Features of 3rd degree via forward selection
X_f_3_f = X_f_3[forward_selection(X_f_3,Y,0.01)]

In [ ]:
# Reducing the Reduced Dataset with Polynomial Features of 3rd degree via backward elimination
X_f_3_b = X_f_3[backward_elimination(X_f_3,Y,0.01)]

In [ ]:
# Reducing the Reduced Dataset with Polynomial Features of 3rd degree via stepwise selection
X_f_3_s = X_f_3[stepwise_selection(X_f_3,Y,0.01,0.01)]

__5.5. Running MLR models and storing results__

In [ ]:
CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_3deg_reduced_f':cv_scores(X_f_3_f,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_3deg_reduced_b':cv_scores(X_f_3_b,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_3deg_reduced_s':cv_scores(X_f_3_s,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS

## 6. CONCLUSION

The best model (error wise) is produced when we expand the original dataset by adding Polynomial Features of 2nd degree and then reducing it via backward elimination.

The rest of the models do not show any remarkable improvement.

It is also obvious that most models that contain predictors with Polynomial Features of 3rd degree are worse that the rest.

__Saving the Results__

In [ ]:
CV_RESULTS.to_csv(r'PolynomialRegressionModels_CV_Results.csv', index=True, index_label = 'Metric')